# AWS Bedrock AgentCore Gateway ターゲット用のベアラートークンインジェクションパターン

## 概要

お客様は、MCP クライアント（Cyphr、Amazon Q）が AgentCore Gateway を通じてバックエンドターゲット（Lambda 関数または RESTful サービス）によって公開されるツール（例：Asana 関数）を呼び出せるようにしたいと考えています。重要な要件は、下流のサービスとの認証のために MCP クライアントから動的なベアラートークンをターゲットに注入することです。主な課題は、AgentCore Gateway の必須アウトバウンド認証要件であり、これが動的ヘッダーインジェクションと競合する可能性があります。

以下のアーキテクチャ図は、AgentCore アウトバウンド認証メカニズムと競合することなく認証トークン（例：「Bearer actual-auth-token」）を注入するために提案するソリューションを示しています。

<img src="./images/token-injection-architecture.png" alt="トークンインジェクションパターンを示すアーキテクチャ図" title="トークンインジェクションパターンを示すアーキテクチャ図" />

このソリューションでは、MCP クライアントがツールで使用する認証トークンを取得し、ペイロードのパラメータとして AgentCore Gateway に渡します。その後、カスタムヘッダーとしてツールに渡されます。

### 前提条件のデプロイ

このソリューションは以下のコンポーネントをデプロイします：
- インバウンド認証用の Cognito ユーザープール
- Lambda 統合を持つ API Gateway
- AgentCore アウトバウンド認証用の API キー

In [ ]:
cd prerequisites/agentcore-components

In [ ]:
!bash prereq.sh

## API Gateway をターゲットとして AgentCore Gateway を作成

### ステップ 1: 依存関係のインストールとインポート

始める前に、このラボの前提条件をインストールしましょう

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得
if "__file__" in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例：Jupyter）

print(f"current_dir を {current_dir} に設定しました")

# utils.py を含むディレクトリに移動（2つ上の階層、"04-bearer-token-injection" へ）
utils_dir = os.path.abspath(os.path.join(current_dir, "../.."))

# sys.path に追加
sys.path.insert(0, utils_dir)

import utils

In [ ]:
# 必要なパッケージをインストール
%pip install -U -r ../../requirements.txt -q

### ステップ 2: AgentCore Gateway を作成

以下の Python コードは：
- AgentCore Gateway を作成します
- 以前に作成した APIGateway を API_KEY と共にターゲットとして追加します。

In [ ]:
# agentcore_gateway_creation モジュールから関数をインポート
import sys
import os

# モジュールをインポートするために現在のディレクトリを Python パスに追加
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

from agentcore_gateway_creation import create_agentcore_gateway

# ステップ 1: AgentCore Gateway を作成
print("🚀 AgentCore Gateway を作成中...")
gateway = create_agentcore_gateway()
print(f"✅ Gateway を作成しました。ID: {gateway['id']}")
print(f"Gateway URL: {gateway['gateway_url']}")

### ステップ 3: AgentCore Gateway に APIGateway をターゲットとして追加

まず AgentCore Gateway が「READY」状態であることを確認しましょう。

In [ ]:
import time
import boto3

# Gateway が ACTIVE 状態になるまで待機
gateway_client = boto3.client("bedrock-agentcore-control")
while True:
    response = gateway_client.get_gateway(gatewayIdentifier=gateway["id"])
    status = response["status"]
    print(f"Gateway ステータス: {status}")
    if status == "READY":
        break
    elif status == "FAILED":
        raise Exception("Gateway の作成に失敗しました")
    time.sleep(10)  # 再度確認する前に 10 秒待機

次に、セットアップを完了するためにゲートウェイターゲットを追加しましょう：

In [ ]:
from agentcore_gateway_creation import add_gateway_target

# ステップ 2: Gateway ターゲットを追加
print("🎯 Gateway ターゲットを追加中...")
add_gateway_target(gateway["id"])
print("✅ Gateway ターゲットの設定が完了しました！")

このターゲットを定義するために使用された OpenAPI 仕様を見てみましょう：

<img src="./images/bearer-token-in-api-spec.png" alt="API 仕様でヘッダーとして定義されたベアラートークン" title="API 仕様でヘッダーとして定義されたベアラートークン" />

トークンはスキーマでヘッダーパラメータとして定義され、HTTP ヘッダー ```X-Asana-Token``` で転送されます。これは一例であり、統合のためにこのようなヘッダーを定義できます。

HTTP ヘッダーは認証トークンの最も一般的なパターンです。

### ステップ 4: エンドツーエンドのフローをテスト

MCP クライアントから ``` tools/list ``` を呼び出すエンドツーエンドのフローをテストしましょう。

In [ ]:
import os
import boto3
import time
from botocore.exceptions import ClientError
import utils
import uuid
import json
import requests

# ステップ 1: MCP クライアントが AgentCore で認証するためのインバウンド認証トークンを取得
token_url = utils.get_ssm_parameter("/app/asana/demo/agentcoregwy/cognito_token_url")
client_id = utils.get_ssm_parameter("/app/asana/demo/agentcoregwy/machine_client_id")
client_secret = utils.get_cognito_client_secret()
inbound_auth_token = utils.fetch_access_token(client_id, client_secret, token_url)
print("アクセストークンを受信しました")

# ステップ 2: Gateway 設定を取得してペイロードを準備
GATEWAY_MCP_URL = gateway_url = (
    f"https://{utils.get_ssm_parameter('/app/asana/demo/agentcoregwy/gateway_id')}.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp"
)

SESSION_ID = str(uuid.uuid4())
headers = {
    "Authorization": f"Bearer {inbound_auth_token}",  # インバウンド認証用
    "Content-Type": "application/json",
    "Mcp-Session-Id": SESSION_ID,
}

# ステップ 3: tools/list を呼び出して利用可能なツールを取得
list_body = {"jsonrpc": "2.0", "id": "list-1", "method": "tools/list"}

list_response = requests.post(GATEWAY_MCP_URL, headers=headers, json=list_body)
print(f"tools/list ステータス: {list_response.status_code}")
print("利用可能なツール:")
print(json.dumps(list_response.json(), indent=2))

### ステップ 5: MCP クライアントから AgentCore Gateway ターゲットへのベアラートークンインジェクション

次に、パラメータでベアラートークン ``` Bearer ASANA-TOKEN ``` を渡し（``` ASANA-TOKEN ``` は MCP クライアントが取得した実際のトークンに置き換えます）、上記の ```tools/list``` レスポンスからツール ```AgentCoreGwyAPIGatewayTarget___asanaInvoke``` を呼び出します。

In [ ]:
# ステップ 1: MCP クライアントが AgentCore で認証するためのインバウンド認証トークンを取得
token_url = utils.get_ssm_parameter("/app/asana/demo/agentcoregwy/cognito_token_url")
client_id = utils.get_ssm_parameter("/app/asana/demo/agentcoregwy/machine_client_id")
client_secret = utils.get_cognito_client_secret()
inbound_auth_token = utils.fetch_access_token(client_id, client_secret, token_url)
print("アクセストークンを受信しました")

# ステップ 2: Gateway 設定を取得してペイロードを準備
GATEWAY_MCP_URL = gateway_url = (
    f"https://{utils.get_ssm_parameter('/app/asana/demo/agentcoregwy/gateway_id')}.gateway.bedrock-agentcore.us-east-1.amazonaws.com/mcp"
)

SESSION_ID = str(uuid.uuid4())
headers = {
    "Authorization": f"Bearer {inbound_auth_token}",  # インバウンド認証用
    "Content-Type": "application/json",
    "Mcp-Session-Id": SESSION_ID,
}

# ステップ 3: Asana トークンを引数に含めて tools/call を呼び出し（更新されたスキーマに従ってヘッダーとして転送）
call_body = {
    "jsonrpc": "2.0",
    "id": "call-1",
    "method": "tools/call",
    "params": {
        "name": "AgentCoreGwyAPIGatewayTarget___asanaInvoke",  # tools/list からの正確な名前
        "arguments": {
            "tool_name": "createTask",
            "X-Asana-Token": "Bearer ASANA-TOKEN",  # 実際の Asana ベアラートークンをここに（ヘッダーとして転送）
            "name": "MCP からのテストタスク",
            "notes": "これはテスト用の説明です",
            "project": "your-project-gid",  # 実際のプロジェクト GID に置き換え
        },
    },
}

call_response = requests.post(GATEWAY_MCP_URL, headers=headers, json=call_body)
print(f"tools/call ステータス: {call_response.status_code}")
print("ツール呼び出しレスポンス:")
print(json.dumps(call_response.json(), indent=2))

### ステップ 6: トークンインジェクションの検証

CloudWatch のログを確認してトークンインジェクションを検証しましょう。前提条件で作成された ```AsanaIntegrationStackInfra``` CloudFormation スタックに移動します：

```CloudFormation -> Stacks -> AsanaIntegrationStackInfra``` を開き、以下に示すように Lambda 関数を探します：
<img src="./images/AgentCoreGwyAsanaIntegrationDemo_function_resource.png" alt="AgentCoreGwyAsanaIntegrationDemo 関数リソース" title="AgentCoreGwyAsanaIntegrationDemo 関数リソース"/>


Lambda 関数を別のウィンドウで開き、以下のスクリーンショットに示すように ```View CloudWatch Logs``` をクリックします：<img src="./images/AgentCoreGwyAsanaIntegrationDemo_function_log_group.png" alt="AgentCoreGwyAsanaIntegrationDemo 関数ロググループ" title="AgentCoreGwyAsanaIntegrationDemo 関数ロググループ"/>


以下のスクリーンショットは、AgentCore Gateway 経由での ```AgentCoreGwyAPIGatewayTarget___asanaInvoke``` ツール呼び出しのログを示しています。OpenAPI 仕様で定義されたヘッダー ``` X-Asana-Token ``` で、``` Bearer トークン ``` が MCP クライアントから AgentCore Gateway を経由して Lambda 関数まで渡されたことがわかります。

<img src="./images/Lambda-log-received-headers.png" alt="ヘッダーとして注入されたベアラートークン" title="ヘッダーとして注入されたベアラートークン" />


これは、MCP クライアントが取得した認証トークンを、AgentCore アウトバウンド認証と競合することなく、Lambda 関数（または他のツール）が Asana のようなサードパーティ SaaS 統合との認証に使用できることを示しています。


MCP クライアントからの残りのペイロードは、以下に示すようにイベントのボディで Lambda 関数から利用可能です：
<img src="./images/Lambda-log-received-body.png" alt="ペイロードボディ" title="ペイロードボディ" />

## クリーンアップ

### AgentCore Gateway を削除

In [ ]:
import utils
import boto3

gateway_client = boto3.client("bedrock-agentcore-control")

gateway_id = "agentcore-gw-asana-integration"
utils.delete_gateway(gateway_client, gateway_id)

### CloudFormation スタックを削除

In [ ]:
cd 02-AgentCore-gateway/04-bearer-token-injection

In [ ]:
!bash clean_up.sh